# Assignment 3 (Memory)


In [19]:
from langchain.chat_models.openai import ChatOpenAI
from langchain.chat_models.ollama import ChatOllama
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import (
    FewShotChatMessagePromptTemplate,
    FewShotPromptTemplate,
)
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser
from langchain.memory import ConversationBufferMemory


# 챗 지피티
chat = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o-mini-2024-07-18",
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# 로컬 LLM (ollama)
chat_ollama = ChatOllama(
    temperature=0.1,
    # model="mixtral:instruct",
    # model="stablelm2:latest",
    model="gemma2:2b",
    # model="gemma2:latest",
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


example = [
    {"movie": "탑건", "answer": "🛩️👨‍✈️🔥"},
    {"movie": "대부", "answer": "👨‍👨‍👦🔫🍝"},
    {"movie": "쥬라기 공원", "answer": "🦖🏞️🚗"},
    {"movie": "해리 포터", "answer": "🧙‍♂️⚡🏰"},
    {"movie": "토이 스토리", "answer": "🤠🚀🧸"},
    {"movie": "타이타닉", "answer": "🚢💏🧊"},
    {"movie": "인셉션", "answer": "💤🌀🕰️"},
    {"movie": "어벤져스", "answer": "🦸‍♂️🦸‍♀️🌌"},
    {"movie": "라이온 킹", "answer": "🦁👑🌅"},
    {"movie": "겨울왕국", "answer": "❄️👸🎶"},
    {"movie": "매트릭스", "answer": "🕶️💊🐇"},
    {"movie": "스타워즈", "answer": "⭐🚀🤖"},
]


memory = ConversationBufferMemory(
    memory_key="history",
    return_messages=True,
)

# memory = ConversationSummaryBufferMemory(
#     llm=chat,
#     max_token_limit=120,
#     memory_key="history",
#     return_messages=True,
# )


def load_memory(_):
    return memory.load_memory_variables({})["history"]


output_parser = StrOutputParser()

example_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            "영화 제목을 말하면 영화를 설명하는 이모티콘으로 답변해줘. 영화제목은 {movie}야",
        ),
        ("ai", "{movie} : {answer}\n"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=example,
)


explain_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 영화에 대해 이모티콘 3개로 설명할 수 있어."),
        example_prompt,
        ("human", "{movie}"),
    ]
)

find_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 이제까지의 대화내용을 보고 나에게 말해 줄 수 있어."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


# 체인 생성 (중간에 chat_ollama를 넣어서 로컬 LLM을 사용함 -> chat으로 바꾸면 오픈AI 사용)
explain_chain = explain_prompt | chat_ollama | output_parser

find_chain = (
    RunnablePassthrough.assign(history=load_memory)
    | find_prompt
    | chat_ollama
    | output_parser
)


def invoke_chain(question):
    result = explain_chain.invoke({"movie": question})
    memory.save_context(
        {"input": question},
        {"output": result},
    )
    print("")

In [20]:
invoke_chain("로보캅")

invoke_chain("석양의 무법자")

invoke_chain("핑크팬더")

[INST] 로보캅으로 인해 이모티콘이 찾을 수 없습니다. 가능한 시간에 다시 시도해주세요. [</INST]>
네, 이해풍니다. "석양의 무법자"이라는 영화标题입니다. 감동과 스토리를 담은 작품으로 알려주시면 감사하겠습니다!
한국의 "핑크팬더"을 대하면 다음과 같습니다:

핑크팬더: 🎮🌟💡

이 모티션은 핑크팬더를 나타내며, 게임을 즐기고 있습니다. 게임은 흥미로운 능력과 기술을 사용하여 경험을 쌓고 있습니다. 이 모티션은 핑크팬더를 나타내며, 성장 및 발전을 추구합니다.


In [21]:
find_chain.invoke({"question": "내가 첫번째에 물어본 영화제목은 뭐야?"})
print("")
find_chain.invoke({"question": "내가 두번째에 물어본 영화제목은 뭐야?"})
print("")
find_chain.invoke({"question": "내가 세번째에 물어본 영화제목은 뭐야?"})
print("")
memory.load_memory_variables({})["history"]

네, 이해풍니다. "석양의 무법자"이라는 영화标题입니다. 감동과 스토리를 담은 작품으로 알려주시면 감사하겠습니다!
네, 이해풍니다. "석양의 무법자"이라는 영화标题입니다. 감동과 스토리를 담은 작품으로 알려주시면 감사하겠습니다!
네, 이해풍니다. "석양의 무법자"이라는 영화标题입니다. 감동과 스토리를 담은 작품으로 알려주시면 감사하겠습니다!


[HumanMessage(content='로보캅'),
 AIMessage(content='[INST] 로보캅으로 인해 이모티콘이 찾을 수 없습니다. 가능한 시간에 다시 시도해주세요. [</INST]>'),
 HumanMessage(content='석양의 무법자'),
 AIMessage(content='네, 이해풍니다. "석양의 무법자"이라는 영화标题입니다. 감동과 스토리를 담은 작품으로 알려주시면 감사하겠습니다!'),
 HumanMessage(content='핑크팬더'),
 AIMessage(content='한국의 "핑크팬더"을 대하면 다음과 같습니다:\n\n핑크팬더: 🎮🌟💡\n\n이 모티션은 핑크팬더를 나타내며, 게임을 즐기고 있습니다. 게임은 흥미로운 능력과 기술을 사용하여 경험을 쌓고 있습니다. 이 모티션은 핑크팬더를 나타내며, 성장 및 발전을 추구합니다.')]